<a href="https://colab.research.google.com/github/lmarchyok/ICEsat-2/blob/main/Solar_Elevation_vs_Saturation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# connect to google drive

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# import dependencies

import pandas as pd
import os
import numpy as np
import zipfile
from zipfile import BadZipFile
import h5py
import matplotlib.pylab as plt
from IPython.utils import strdispatch

In [27]:
# one big class for all of our data processing and visualization methods

class ICEsat_2_visualizer: 

  all = []

  # grab the variables we want to plot against one another at the instantiation of the class 

  def __init__(self, data_dir: str, laser_id: str, var1: str, var2: str):
    self.data_dir = data_dir
    self.laser_id = laser_id
    self.var1 = var1
    self.var2 = var2

    ICEsat_2_visualizer.all.append(self)

  def __repr__(self): 
    return f'{self.data_id}'

  # unzip the zipfiles and move folder containing h5 files to 'oregon data' folder 

  def zip_extraction(self):

    destination_dir = '/content/drive/MyDrive/ICEsat-2/ICEsat-2_data_downloads/OR'

    zp = zipfile.ZipFile(self.data_dir)
    zp.extractall(destination_dir)
    
    for i in os.listdir(destination_dir):

      if i == 'Oregon_Data':
        continue
      elif i == 'IceSat2_Data_OR.pptx':
        continue
      elif i == '._IceSat2_Data_OR.pptx':
        continue
      elif i in ICEsat_2_visualizer.all:
        continue
      else:
        self.data_id = i
        final = destination_dir + '/' + i
        finaldir = os.listdir(final)[0]

    return final + '/' + finaldir 

  # extract data from h5 file

  def h5_extraction(self):

    with h5py.File(self.zip_extraction(), 'r') as f1:
        laser = f1.get(self.laser_id)
        laser_data = laser.get('geolocation')
        var1 = laser_data.get(self.var1)
        var2 = laser_data.get(self.var2)
        var1_arr = np.array(var1)

        # delete extreme outlier data points for solar_elevation
        if self.var1 == 'solar_elevation':
          var1_arr_cur = np.delete(var1_arr, (var1_arr > 180))

          var2_arr = np.array(var2)

          for i in np.where(var1_arr > 180):

            list_of_bad_datapoints = []

            if len(i) > 0:
              for j in i:
                list_of_bad_datapoints.append(j)
              var2_arr_cur = np.delete(var2_arr, list_of_bad_datapoints)
            else:
              var2_arr_cur = var2_arr
        
          print(var1_arr_cur)
          print(var2_arr_cur)

        f1.close()
    
    return (var1_arr_cur, var2_arr_cur)

  # create a figures folder and visualize data using matlab-style plot 

  def plot_sat_vs_sol_elev(self, selims='auto'):

    if selims=='auto':
          lower = np.nanmin(self.h5_extraction()[1])
          upper = np.nanmax(self.h5_extraction()[1])
          serange = upper - lower
          selims = [lower - 0.2*serange, upper + 0.1*serange]

    lat_lower = np.nanmin(self.h5_extraction()[0])
    lat_upper = np.nanmax(self.h5_extraction()[0])
    latrange = lat_upper - lat_lower
    latlims = [lat_lower - 0.2 * latrange, lat_upper + 0.1 * latrange]

    fig = plt.figure(figsize=(9, 6), dpi=80, facecolor='w', edgecolor='k')
    ax = fig.add_subplot(111)
    solar_elevation = self.h5_extraction()[0]
    saturation = self.h5_extraction()[1]
    ax.set_title(f'{self.var1} vs {self.var2} over 700m ({self.laser_id}')
    ax.set_xlabel('Solar Elevation (degrees)')
    ax.set_ylabel('Saturation (percentage)')

    atl03_plot = ax.scatter(solar_elevation, saturation, s=10,c='k',edgecolors='none',label=self.laser_id)
    ax.legend(handles=[atl03_plot], loc='lower left')
    ax.set_xlim(latlims)
    ax.set_ylim(selims)

    if not os.path.exists(f'figs/{self.data_id}'):
        os.makedirs(f'figs/{self.data_id}')
    fn = f'figs/{self.data_id}/{self.var1}_vs_{self.var2}_{self.laser_id}.png'
    plt.savefig(fn, dpi=150)
    plt.close()

In [17]:
  # main function loops through the data directory, and plots solar_elevation vs full_sat_fract for each laser
  
  def main():
    oregon_datapath = '/content/drive/MyDrive/ICEsat-2/ICEsat-2_data_downloads/OR/Oregon_Data/'
    beams = ['gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r']

    for zipfile in os.listdir(oregon_datapath):
      if zipfile[:2] == '._':
        continue
      else:
        print(zipfile)
        for beam in beams:
          visualizer = ICEsat_2_visualizer((oregon_datapath + zipfile), beam, 'solar_elevation', 'full_sat_fract')
          visualizer.zip_extraction()
          visualizer.h5_extraction()
          visualizer.plot_sat_vs_sol_elev()
      break

In [ ]:
if __name__ == '__main__':
  main()